In [1]:
import pandas as pd
import datetime as date
import re

## Extract

In [2]:
#Creating a list of years I want data for
year = date.datetime.today().year-1
years = [year - i for i in range(20)]


In [3]:
#Pulling all of my data from a website. I am pulling two seperate webpages(offense and defense)
team_stats = []
for y in years:
    
    #Two html addresses
    team_offense = pd.read_html(f'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr={y}&type=reg&cat=T&group=O&conf=')
    team_defense = pd.read_html(f'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr={y}&type=reg&cat=T&group=D&conf=')
    
    team_o = team_offense[0]
    team_d = team_defense[0]
    
    #merging the the offensive and defensive dataframes together on their team name
    merged_inner = pd.merge(left=team_o, right=team_d, left_on='Team', right_on='Team')
    
    #This code is to handle a problem where the team name was connected to the city name and repeating itself
    city=[]
    team_name=[]
    for j in range(0,len(merged_inner.Team)):
        team_temp = merged_inner.Team[j].split(" ")
        if len(team_temp)==4:
            city.append(team_temp[0]+" "+team_temp[1])
            if team_temp[2]!="49ersSan":
                caps=re.findall('[A-Z][^A-Z]*',team_temp[2])
                team_name.append(caps[0])
            else:
                caps="49ers"
                team_name.append(caps)
            #team_name.append(team_temp[2])
            
        elif len(team_temp)==2:
            city.append(team_temp[0])
            caps=re.findall('[A-Z][^A-Z]*',team_temp[1])
            team_name.append(caps[0])
            #team_name.append(team_temp[1])
            
        else:
            print('')
    
    #cleaned up the columns
    merged_inner['city']=city
    merged_inner['Team Name']=team_name
    merged_inner['year']= y
    merged_inner=merged_inner.drop(columns=["Team"])
    
    team_stats.append(merged_inner)
    
team_stats



[    Gms_x  Tot Pts_x  Pts/G_x  RushYds_x  RYds/G_x  PassYds_x  PYds/G_x  \
 0      16        434     27.1       2153     134.6       4751     296.9   
 1      16        531     33.2       3296     206.0       3225     201.6   
 2      16        458     28.6       1521      95.1       4845     302.8   
 3      16        479     29.9       2305     144.1       3792     237.0   
 4      16        381     23.8       1361      85.1       4714     294.6   
 5      16        451     28.2       1569      98.1       4498     281.1   
 6      16        394     24.6       1499      93.7       4499     281.2   
 7      16        405     25.3       2200     137.5       3791     236.9   
 8      16        458     28.6       1738     108.6       4244     265.2   
 9      16        337     21.1       1453      90.8       4426     276.6   
 10     16        313     19.6       1893     118.3       3926     245.4   
 11     16        402     25.1       2223     138.9       3582     223.9   
 12     16  

## Transform

In [4]:
#combining all of the dataframes for each year into a single dataframe
combined_stats= pd.concat(team_stats)


In [5]:
#dropping unecessary columns
combined_stats=combined_stats.drop(columns=['Gms_y','RushYds_x','PassYds_x','TotYds_x','RushYds_y','PassYds_y','TotYds_y'],errors='ignore')
combined_stats

,Gms_x,Tot Pts_x,Pts/G_x,RYds/G_x,PYds/G_x,Yds/G_x,Tot Pts_y,Pts/G_y,RYds/G_y,PYds/G_y,Yds/G_y,city,Team Name,year
0,16,434,27.1,134.6,296.9,431.5,321,20.1,103.5,223.5,327.0,Dallas,Cowboys,2019
1,16,531,33.2,206.0,201.6,407.6,282,17.6,93.4,207.2,300.6,Baltimore,Ravens,2019
2,16,458,28.6,95.1,302.8,397.9,449,28.1,73.8,270.1,343.9,Tampa Bay,Buccaneers,2019
3,16,479,29.9,144.1,237.0,381.1,310,19.4,112.6,169.2,281.8,San Francisco,49ers,2019
4,16,381,23.8,85.1,294.6,379.7,399,24.9,110.9,244.9,355.8,Atlanta,Falcons,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,16,307,19.2,109.2,167.2,276.4,307,19.2,113.9,200.6,314.6,Detroit,Lions,2000
27,16,269,16.8,66.4,202.4,268.8,440,27.5,88.9,221.1,309.9,San Diego,Chargers,2000
28,16,185,11.6,144.6,121.6,266.2,359,22.4,120.3,222.6,342.9,Cincinnati,Bengals,2000
29,16,252,15.8,75.9,173.8,249.6,413,25.8,123.9,226.5,350.4,Atlanta,Falcons,2000


In [6]:
#Dropping columns again. These two columns would not work the previous way
combined_stats=combined_stats.drop([combined_stats.columns[1] , combined_stats.columns[6]] ,  axis='columns')
combined_stats

,Gms_x,Pts/G_x,RYds/G_x,PYds/G_x,Yds/G_x,Pts/G_y,RYds/G_y,PYds/G_y,Yds/G_y,city,Team Name,year
0,16,27.1,134.6,296.9,431.5,20.1,103.5,223.5,327.0,Dallas,Cowboys,2019
1,16,33.2,206.0,201.6,407.6,17.6,93.4,207.2,300.6,Baltimore,Ravens,2019
2,16,28.6,95.1,302.8,397.9,28.1,73.8,270.1,343.9,Tampa Bay,Buccaneers,2019
3,16,29.9,144.1,237.0,381.1,19.4,112.6,169.2,281.8,San Francisco,49ers,2019
4,16,23.8,85.1,294.6,379.7,24.9,110.9,244.9,355.8,Atlanta,Falcons,2019
...,...,...,...,...,...,...,...,...,...,...,...,...
26,16,19.2,109.2,167.2,276.4,19.2,113.9,200.6,314.6,Detroit,Lions,2000
27,16,16.8,66.4,202.4,268.8,27.5,88.9,221.1,309.9,San Diego,Chargers,2000
28,16,11.6,144.6,121.6,266.2,22.4,120.3,222.6,342.9,Cincinnati,Bengals,2000
29,16,15.8,75.9,173.8,249.6,25.8,123.9,226.5,350.4,Atlanta,Falcons,2000


In [7]:
#Renaming columns
combined_stats=combined_stats.rename(columns={
    "Gms_x": "games", 
    "Pts/G_x": "points_scored_gm",
    "RYds/G_x": "rush_yds_gm" ,
    "PYds/G_x": "pass_yds_gm",
    "Yds/G_x": "total_yds_gm",
    "Pts/G_y":"points_allowed_gm",
    "RYds/G_y": "rush_yds_allowed_gm",
    "PYds/G_y": "pass_yds_allowed_gm",
    "Yds/G_y": "yds_allowed_gm",
    "Team Name":"team_name"})

In [8]:
combined_stats

,games,points_scored_gm,rush_yds_gm,pass_yds_gm,total_yds_gm,points_allowed_gm,rush_yds_allowed_gm,pass_yds_allowed_gm,yds_allowed_gm,city,team_name,year
0,16,27.1,134.6,296.9,431.5,20.1,103.5,223.5,327.0,Dallas,Cowboys,2019
1,16,33.2,206.0,201.6,407.6,17.6,93.4,207.2,300.6,Baltimore,Ravens,2019
2,16,28.6,95.1,302.8,397.9,28.1,73.8,270.1,343.9,Tampa Bay,Buccaneers,2019
3,16,29.9,144.1,237.0,381.1,19.4,112.6,169.2,281.8,San Francisco,49ers,2019
4,16,23.8,85.1,294.6,379.7,24.9,110.9,244.9,355.8,Atlanta,Falcons,2019
...,...,...,...,...,...,...,...,...,...,...,...,...
26,16,19.2,109.2,167.2,276.4,19.2,113.9,200.6,314.6,Detroit,Lions,2000
27,16,16.8,66.4,202.4,268.8,27.5,88.9,221.1,309.9,San Diego,Chargers,2000
28,16,11.6,144.6,121.6,266.2,22.4,120.3,222.6,342.9,Cincinnati,Bengals,2000
29,16,15.8,75.9,173.8,249.6,25.8,123.9,226.5,350.4,Atlanta,Falcons,2000


In [9]:
combined_stats['team_name'] = combined_stats['team_name'].astype(str)
combined_stats['city'] = combined_stats['city'].astype(str)
combined_stats['team'] = combined_stats['city'].astype(str)+' '+combined_stats['team_name']

In [10]:
combined_stats=combined_stats[['team','year','points_scored_gm','rush_yds_gm','pass_yds_gm','total_yds_gm','points_allowed_gm','rush_yds_allowed_gm','pass_yds_allowed_gm','yds_allowed_gm']]
combined_stats

,team,year,points_scored_gm,rush_yds_gm,pass_yds_gm,total_yds_gm,points_allowed_gm,rush_yds_allowed_gm,pass_yds_allowed_gm,yds_allowed_gm
0,Dallas Cowboys,2019,27.1,134.6,296.9,431.5,20.1,103.5,223.5,327.0
1,Baltimore Ravens,2019,33.2,206.0,201.6,407.6,17.6,93.4,207.2,300.6
2,Tampa Bay Buccaneers,2019,28.6,95.1,302.8,397.9,28.1,73.8,270.1,343.9
3,San Francisco 49ers,2019,29.9,144.1,237.0,381.1,19.4,112.6,169.2,281.8
4,Atlanta Falcons,2019,23.8,85.1,294.6,379.7,24.9,110.9,244.9,355.8
...,...,...,...,...,...,...,...,...,...,...
26,Detroit Lions,2000,19.2,109.2,167.2,276.4,19.2,113.9,200.6,314.6
27,San Diego Chargers,2000,16.8,66.4,202.4,268.8,27.5,88.9,221.1,309.9
28,Cincinnati Bengals,2000,11.6,144.6,121.6,266.2,22.4,120.3,222.6,342.9
29,Atlanta Falcons,2000,15.8,75.9,173.8,249.6,25.8,123.9,226.5,350.4


In [18]:
abrv = pd.read_csv('../nate/tables/teams.csv')

abrv.head()

,TM,Team
0,ARI,Arizona Cardinals
1,ATL,Atlanta Falcons
2,BAL,Baltimore Ravens
3,BUF,Buffalo Bills
4,CAR,Carolina Panthers


## Load

In [11]:
from sqlalchemy import create_engine

In [12]:
combined_stats.to_csv('Team_Statistics.csv')

In [24]:
# Create connection
connection_string = "postgres:postgres@localhost:5432/nfl_db"
engine = create_engine(f'postgresql://{connection_string}')

In [25]:
combined_stats.to_sql(name='team_stats', con=engine, if_exists='append', index=False)